In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
import glob
import os
from cv2 import resize
from mat_track.main import mat_track
from utils import overlay, binarize
from models.budNET.model import BudNET
from track.lineage import LineageConstruction
from utils import overlay, binarize, rescaleByFactor
from track.utils import get_cell_bbox
import matplotlib.pyplot as plt
import pandas as pd

/Users/berk/opt/miniconda3/envs/budNET/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-31 21:50:01.573877: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
cmap = plt.get_cmap("Greens", 11)
cmap(7.3)

(0.0, 0.26666666666666666, 0.10588235294117647, 1.0)

In [17]:
round(0.65,1)*10

7.0

In [3]:
cp_masks = imread("test/test_images/test_tracked.tif")
phase = imread("test/test_images/test_phase.tif")
necks = imread("test/test_images/test_buds.tif")
cp_masks.shape, phase.shape, necks.shape

((10, 520, 692), (10, 520, 692), (10, 520, 692))

In [4]:
lng = LineageConstruction(cp_masks, necks, backskip = 2, forwardskip = 5)
data = lng.computeLineages()


computing lineages for 140 cells


100%|██████████| 140/140 [00:01<00:00, 80.85it/s]


In [5]:
daughterDF, motherDF = data

In [6]:
motherDF

,cell,birth,death,mother,confidence
0,1,0,9,,0.00
1,2,0,9,1,1.00
2,3,0,9,,0.00
3,4,0,9,,0.00
4,5,0,9,,0.00
...,...,...,...,...,...
135,136,8,9,,0.00
136,137,9,9,10,0.80
137,138,9,9,77,0.99
138,139,9,9,89,1.00


In [7]:
groups = motherDF.groupby(["mother", "birth"])
result = groups.filter(lambda x: len(x) > 1)
result.index.tolist()

[0,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 17,
 18,
 19,
 20,
 21,
 22,
 24,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 42,
 44,
 45,
 48,
 50,
 51,
 52,
 53,
 54,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 67,
 69,
 70,
 71,
 74,
 76,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 87,
 88,
 89,
 90,
 91,
 93,
 94,
 95,
 96,
 97,
 98,
 103,
 105]

In [8]:
motherDF.to_csv("/Users/berk/Downloads/test_lineage.csv")

In [9]:
daughterDF

,1,2,3,4,5,6,7,8,9,10,...,131,132,133,134,135,136,137,138,139,140
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
137,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
138,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
139,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
motherDF.iloc[0,3] == ''

True